# Init

In [1]:
!neofetch

            .-/+oossssoo+/-.
        `:+ssssssssssssssssss+:`
      -+ssssssssssssssssssyyssss+-
    .ossssssssssssssssssdMMMNysssso.
   /ssssssssssshdmmNNmmyNMMMMhssssss/
  +ssssssssshmydMMMMMMMNddddyssssssss+
 /sssssssshNMMMyhhyyyyhmNMMMNhssssssss/
.ssssssssdMMMNhsssssssssshNMMMdssssssss.
+sssshhhyNMMNyssssssssssssyNMMMysssssss+
ossyNMMMNyMMhsssssssssssssshmmmhssssssso
ossyNMMMNyMMhsssssssssssssshmmmhssssssso
+sssshhhyNMMNyssssssssssssyNMMMysssssss+
.ssssssssdMMMNhsssssssssshNMMMdssssssss.
 /sssssssshNMMMyhhyyyyhdNMMMNhssssssss/
  +sssssssssdmydMMMMMMMMddddyssssssss+
   /ssssssssssshdmNNNNmyNMMMMhssssss/
    .ossssssssssssssssssdMMMNysssso.
      -+sssssssssssssssssyyyssss+-
        `:+ssssssssssssssssss+:`
            .-/+oossssoo+/-.
alz@alz-System-Product-Name 
--------------------------- 
OS: Ubuntu 20.04.1 LTS x86_64 
Kernel: 5.4.0-47-generic 
Uptime: 22 hours, 14 mins 
Packages: 2062 (dpkg), 6 (snap) 
Shell: bash 5.0.17 
Resolution: 1920x1080 
DE: Plasma 
WM: KWin 
WM Theme: Br

In [2]:
from dask.distributed import Client, LocalCluster

worker_kwargs = {
#     'memory_limit': '1G',
#     'memory_target_fraction': 0.6,
    'memory_spill_fraction': 1,
#     'memory_pause_fraction': 0.8,
#     'memory_terminate_fraction': 0.95,
}

cluster = LocalCluster(processes=False, **worker_kwargs)
client = Client(cluster)
client

Client Scheduler: inproc://192.168.0.102/57012/1 Dashboard: http://192.168.0.102:8787/status,Cluster Workers: 1 Cores: 4 Memory: 33.60 GB


In [3]:
import numpy as np
import dask.array as da

# Задача 1

Постановка задачи:
$$\begin{aligned}
x_i &= x_{i-1} + a, \qquad x \in [1, \, n],\\
x_0 &= b.
\end{aligned}$$

Замкнутая форма
$$x_i = x_0 + ia, \qquad x \in [1, \, n].$$

Векторизация замкнутой формы
$$(x_1, \, x_2, \, \dots, \, x_n) = x_0 + (1, \, 2, \, \dots, \, n) \, a = x_0 + \operatorname*{arange}(1, \, n + 1) \, a.$$

In [6]:
a  = 1.
x0 = 1.
n  = int(1.5e9)

### None

Numpy

In [13]:
x1n = %time x0 + np.arange(1, n + 1) * a
display(x1n)
del x1n

CPU times: user 4.57 s, sys: 2.36 s, total: 6.92 s
Wall time: 6.62 s


array([2.0e+00, 3.0e+00, 4.0e+00, ..., 1.5e+09, 1.5e+09, 1.5e+09])

Dask

In [14]:
x1n = x0 + da.arange(1, n + 1) * a
display(x1n)
_ = %time x1n.compute()
del _

,Array,Chunk
Bytes,12.00 GB,125.00 MB
Shape,"(1500000000,)","(15625000,)"
Count,288 Tasks,96 Chunks
Type,float64,numpy.ndarray


CPU times: user 13.1 s, sys: 7.95 s, total: 21 s
Wall time: 7.88 s


### Sum

Numpy

In [12]:
x1n = %time (x0 + np.arange(1, n + 1) * a).sum()
display(x1n)
del x1n

CPU times: user 5.22 s, sys: 2.39 s, total: 7.61 s
Wall time: 7.26 s


1.12500000225e+18

Dask

In [11]:
x1n = (x0 + da.arange(1, n + 1) * a).sum()
display(x1n)
_ = %time x1n.compute()
del _

dask.array<sum-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

CPU times: user 12.9 s, sys: 6.24 s, total: 19.1 s
Wall time: 5.66 s


# Задача 2

Постановка задачи
$$\begin{aligned}
x_i &= x_{i-1} + a_{i-1}, \qquad i \in [1, \, n], \\
x_0 &= b.
\end{aligned}$$

Замкнутая форма
$$x_i = x_0 + \sum_{j \in [1, \, i]} a_{i-j}, \qquad i \in [1, \, n].$$

Векторизация замкнутой формы
$$(x_1, \, x_2, \, \dots, \, x_n) = x_0 + (a_0, \, a_0 + a_1, \, \dots, \, \sum_1^n a_{n-j}) = x_0 + \operatorname*{cumsum}(a_0, \, a_1, \, \dots, \, a_n).$$

In [11]:
x0 = 1.
n  = int(1e9)

### None

Numpy

In [18]:
a = np.arange(n)
x1n = %time x0 + np.cumsum(a)
display(x1n)
del a, x1n

CPU times: user 4.09 s, sys: 1.53 s, total: 5.62 s
Wall time: 5.39 s


array([1.00000000e+00, 2.00000000e+00, 4.00000000e+00, ...,
       4.99999998e+17, 4.99999998e+17, 5.00000000e+17])

Dask

In [17]:
a = da.arange(n)
x1n = x0 + da.cumsum(a)
display(x1n)
_ = %time x1n.compute()
del _

dask.array<add, shape=(1000000000,), dtype=float64, chunksize=(15625000,), chunktype=numpy.ndarray>

CPU times: user 11.3 s, sys: 6.31 s, total: 17.6 s
Wall time: 8.18 s


### Sum

Numpy

In [16]:
a = np.arange(n)
sum_x1n = %time (x0 + np.cumsum(a)).sum()
display(sum_x1n)
del a, sum_x1n

CPU times: user 4.44 s, sys: 1.69 s, total: 6.12 s
Wall time: 5.87 s


1.666666666666001e+26

Dask

In [15]:
a = da.arange(n)
sum_x1n = (x0 + da.cumsum(a)).sum()
display(sum_x1n)
_ = %time sum_x1n.compute()
del _

dask.array<sum-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

CPU times: user 10.5 s, sys: 5.66 s, total: 16.1 s
Wall time: 6.82 s
